In [6]:
from Dataset import lerCSV,lerXLSX
import warnings
import sys 
import numpy as np
import pickle
import theanets
from sklearn import preprocessing
from random import shuffle
from sklearn.metrics import precision_score,recall_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt
%matplotlib inline

In [7]:
warnings.filterwarnings("ignore")   
matrix = lerXLSX("Banco de Dados - Infarto treinoTeste.xlsx",training=True)

In [8]:
import math

def sigmoid(x):
  x = (x - 0.5) *  10
  return 1 / (1 + math.exp(-x))


In [9]:
k = 10
lista = list(range(0,len(matrix))) 
shuffle(lista)
a = np.asarray(lista) % k

In [10]:
min_max_scaler = preprocessing.MinMaxScaler()
matrixProcessed = min_max_scaler.fit_transform(matrix[:,:4])
matrixProcessed = np.array([[sigmoid(matrixProcessed[y,0]),sigmoid(matrixProcessed[y,1]),sigmoid(matrixProcessed[y,2]),sigmoid(matrixProcessed[y,3])]  for y in range(0,len(matrix[:,0]))])


In [11]:
def KFold(matrix,metric = ["neural network (4,5 tanh,2)",[3, (7, 'tanh'), 2]]):
    k = 8
    
    limit = 3
    
    
    lista = list(range(1,len(matrix))) 
    shuffle(lista)
    a = np.asarray(lista) % k   
    
    scores = np.zeros(k)
    recall = np.zeros(k)
    precision = np.zeros(k)
    average_precision = np.zeros(k)
    
    for fold in range(0,k):
        treinoIndex = np.where(a != fold)[0]
        testeIndex =  np.where(a == fold)[0]        
        
        inputs = np.array([[matrix[y,0],matrix[y,1]]  for y in treinoIndex])
        weights = np.array([int(matrix[y,2] == 1)*2+1 for y in treinoIndex],dtype="int32")        
        outputs = np.array([int(matrix[y,2] == 1) for y in treinoIndex],dtype="int32")
            
        model = theanets.Classifier(metric[1],loss='CrossEntropy',weighted=True)  
        model.train([inputs, outputs,weights],algo='sgd',learning_rate=0.01,momentum=0.9)       
        
        testOutputs = [matrix[y,2] for y in testeIndex] 
        testInputs = np.array([matrix[y,:2]  for y in testeIndex])   
        predicts = model.predict(testInputs)
        scores[fold] = sum([int(predicts[x] == testOutputs[x]) for x in range(len(predicts))]) / len(predicts) 
        recall[fold] = recall_score(testOutputs, predicts)
        precision[fold] = precision_score(testOutputs, predicts)        
        
    
    print("Final:"+ metric[0] + " = Score:%2.2e[+/- %2.2e]"%(np.mean(scores),np.std(scores)))
    print("Final:"+ metric[0] + " = recall:%2.2e[+/- %2.2e]"%(np.mean(recall),np.std(recall)))
    print("Final:"+ metric[0] + " = precision:%2.2e[+/- %2.2e]"%(np.mean(precision),np.std(precision)))       
    return np.mean(scores), np.std(scores)

## Colesterol e Idade

In [15]:
inputOutput = np.array([np.concatenate((matrixProcessed[x,:2],[matrix[x,4] == 1]),axis=0) for x in range(0,len(matrix[:,4]))])
metrics = [ ["neural network (2,5 tanh,2)",[2, (5, 'tanh'), 2]], ["neural network (2,7 tanh,2)",[2, (7, 'tanh'), 2]], ["neural network (2,10 tanh,2)",[2, (10, 'tanh'), 2]], ["neural network (2,15 tanh,2)",[2, (15, 'tanh'), 2]], ["neural network (2,5 tanh,5 tanh,2)",[2,(5, 'tanh'), (5, 'tanh'), 2]]]
for metric in metrics:
    KFold(inputOutput, metric=metric)


Final:neural network (2,5 tanh,2) = Score:6.69e-01[+/- 5.04e-02]
Final:neural network (2,5 tanh,2) = recall:8.31e-01[+/- 5.98e-02]
Final:neural network (2,5 tanh,2) = precision:5.12e-01[+/- 7.16e-02]
Final:neural network (2,7 tanh,2) = Score:6.55e-01[+/- 3.51e-02]
Final:neural network (2,7 tanh,2) = recall:8.29e-01[+/- 6.75e-02]
Final:neural network (2,7 tanh,2) = precision:4.98e-01[+/- 5.06e-02]
Final:neural network (2,10 tanh,2) = Score:6.66e-01[+/- 3.81e-02]
Final:neural network (2,10 tanh,2) = recall:8.43e-01[+/- 6.24e-02]
Final:neural network (2,10 tanh,2) = precision:5.10e-01[+/- 5.08e-02]
Final:neural network (2,15 tanh,2) = Score:6.71e-01[+/- 3.89e-02]
Final:neural network (2,15 tanh,2) = recall:8.28e-01[+/- 6.00e-02]
Final:neural network (2,15 tanh,2) = precision:5.15e-01[+/- 4.66e-02]
Final:neural network (2,5 tanh,5 tanh,2) = Score:6.74e-01[+/- 4.61e-02]
Final:neural network (2,5 tanh,5 tanh,2) = recall:8.42e-01[+/- 6.23e-02]
Final:neural network (2,5 tanh,5 tanh,2) = precis

## Colesterol e Glicemia

In [31]:
inputOutput = np.array([[matrixProcessed[x,0],matrixProcessed[x,2],matrix[x,4] == 1] for x in range(0,len(matrix[:,4]))])
metrics = [ ["neural network (2,5 tanh,2)",[2, (5, 'tanh'), 2]], ["neural network (2,7 tanh,2)",[2, (7, 'tanh'), 2]], ["neural network (2,10 tanh,2)",[2, (10, 'tanh'), 2]], ["neural network (2,15 tanh,2)",[2, (15, 'tanh'), 2]], ["neural network (2,5 tanh,5 tanh,2)",[2,(5, 'tanh'), (5, 'tanh'), 2]]]
for metric in metrics:
    KFold(inputOutput, metric=metric)


Final:neural network (2,5 tanh,2) = Score:5.44e-01[+/- 4.14e-02]
Final:neural network (2,5 tanh,2) = recall:8.81e-01[+/- 7.53e-02]
Final:neural network (2,5 tanh,2) = precision:4.21e-01[+/- 4.81e-02]
Final:neural network (2,7 tanh,2) = Score:5.55e-01[+/- 5.50e-02]
Final:neural network (2,7 tanh,2) = recall:8.52e-01[+/- 6.97e-02]
Final:neural network (2,7 tanh,2) = precision:4.29e-01[+/- 6.78e-02]
Final:neural network (2,10 tanh,2) = Score:5.57e-01[+/- 4.96e-02]
Final:neural network (2,10 tanh,2) = recall:8.80e-01[+/- 4.70e-02]
Final:neural network (2,10 tanh,2) = precision:4.29e-01[+/- 5.15e-02]
Final:neural network (2,15 tanh,2) = Score:5.72e-01[+/- 7.92e-02]
Final:neural network (2,15 tanh,2) = recall:8.82e-01[+/- 5.12e-02]
Final:neural network (2,15 tanh,2) = precision:4.42e-01[+/- 9.65e-02]
Final:neural network (2,5 tanh,5 tanh,2) = Score:5.48e-01[+/- 4.50e-02]
Final:neural network (2,5 tanh,5 tanh,2) = recall:8.72e-01[+/- 6.24e-02]
Final:neural network (2,5 tanh,5 tanh,2) = precis

## Idade e Glicemia

In [32]:
inputOutput = np.array([[matrixProcessed[x,1],matrixProcessed[x,2],matrix[x,4] == 1] for x in range(0,len(matrix[:,4]))])
metrics = [ ["neural network (2,5 tanh,2)",[2, (5, 'tanh'), 2]], ["neural network (2,7 tanh,2)",[2, (7, 'tanh'), 2]], ["neural network (2,10 tanh,2)",[2, (10, 'tanh'), 2]], ["neural network (2,15 tanh,2)",[2, (15, 'tanh'), 2]], ["neural network (2,5 tanh,5 tanh,2)",[2,(5, 'tanh'), (5, 'tanh'), 2]]]
for metric in metrics:
    KFold(inputOutput, metric=metric)

Final:neural network (2,5 tanh,2) = Score:5.95e-01[+/- 3.41e-02]
Final:neural network (2,5 tanh,2) = recall:8.05e-01[+/- 7.12e-02]
Final:neural network (2,5 tanh,2) = precision:4.49e-01[+/- 4.25e-02]
Final:neural network (2,7 tanh,2) = Score:6.10e-01[+/- 2.55e-02]
Final:neural network (2,7 tanh,2) = recall:8.09e-01[+/- 5.20e-02]
Final:neural network (2,7 tanh,2) = precision:4.59e-01[+/- 5.22e-02]
Final:neural network (2,10 tanh,2) = Score:5.90e-01[+/- 3.92e-02]
Final:neural network (2,10 tanh,2) = recall:8.01e-01[+/- 6.00e-02]
Final:neural network (2,10 tanh,2) = precision:4.45e-01[+/- 6.33e-02]
Final:neural network (2,15 tanh,2) = Score:6.05e-01[+/- 4.96e-02]
Final:neural network (2,15 tanh,2) = recall:8.02e-01[+/- 9.88e-02]
Final:neural network (2,15 tanh,2) = precision:4.56e-01[+/- 4.62e-02]
Final:neural network (2,5 tanh,5 tanh,2) = Score:5.91e-01[+/- 4.36e-02]
Final:neural network (2,5 tanh,5 tanh,2) = recall:7.96e-01[+/- 6.87e-02]
Final:neural network (2,5 tanh,5 tanh,2) = precis